# AHP

In [6]:
import pandas as pd
import numpy as np
from ahp import get_priorities, get_relative_consistency_index, recreate_consistent_matrix
from pathlib import Path
import sys

## The hierarchy and the obtained weights

In [40]:

hierarchy = ["comparisons_category.csv", "comparisons_type.csv", "comparisons_alternatives.csv"]

datadir = "../dataset/ahp"

weights = {}
for file_id, file in enumerate(hierarchy):
    data = pd.read_csv(Path(datadir).joinpath(file))

    # Generating pairwise comparison matrix
    for parent in data.parent.unique():
        comparisons = pd.DataFrame(data.loc[data["parent"] == parent, :])
        unique_elements = list(set(comparisons["Element1"].unique().tolist() + comparisons["Element2"].unique().tolist()))

        pairwise_comparisons = pd.DataFrame(index=unique_elements, columns=unique_elements)


        for element in unique_elements:
            pairwise_comparisons.at[element, element] = 1.0
        for comparison_id in comparisons.index:
            el1 = comparisons.at[comparison_id, "Element1"]
            el2 = comparisons.at[comparison_id, "Element2"]
            judgement = comparisons.at[comparison_id, "Judgement"]
            pairwise_comparisons.at[el1, el2] = float(judgement)
            pairwise_comparisons.at[el2, el1]  = 1 / float(judgement)


        comparison_matrix_raw = pairwise_comparisons.to_numpy(dtype=float)
        assert not np.isnan(comparison_matrix_raw).any()
        priorities = get_priorities(comparison_matrix_raw)
        relative_consistency_index = get_relative_consistency_index(comparison_matrix_raw)
        if relative_consistency_index > 0.1:
            raise ValueError(f"Oh no :((( the matrix is too inconsistent RI = {relative_consistency_index}")
            
        # print(comparisons.columns)
        if file_id == 1 and relative_consistency_index > 0:
            print("------------- Example (inconsistent) Comparison Matrix --------------------")
            print(comparison_matrix_raw)

            print("------------- After making it consistent -------------")
            reconstructed = recreate_consistent_matrix(priorities)
            print(reconstructed)

            difference = comparison_matrix_raw - reconstructed
            row_id, col_id = np.unravel_index(np.argmax(difference), difference.shape)
            el1, el2 = pairwise_comparisons.columns[row_id] , pairwise_comparisons.columns[col_id]

            print(f"The biggest discrapency between: {el1} and {el2}")
            print(f"DM value: {pairwise_comparisons.at[el1, el2]}")
            print(f"Value after making it consistent: {reconstructed[row_id,col_id]}")
            
            
        
        for element_id in range(len(pairwise_comparisons.index)):
            element =  pairwise_comparisons.index[element_id]
            weights[element] = priorities[element_id].item()
         

        

------------- Example (inconsistent) Comparison Matrix --------------------
[[1.         9.         4.        ]
 [0.11111111 1.         0.16666667]
 [0.25       6.         1.        ]]
------------- After making it consistent -------------
[[ 1.         12.48050294  2.88449914]
 [ 0.08012498  1.          0.23112042]
 [ 0.34668064  4.32674871  1.        ]]
The biggest discrapency between: EnginePower and Width
DM value: 6.0
Value after making it consistent: 4.326748710922227


In [22]:
hierarchy=pd.read_csv(Path(datadir).joinpath("hierarchy.csv"))

datapoints = pd.read_csv(Path(datadir).joinpath("dataset.csv"), index_col=0)

#copy result but add column score
result = datapoints.copy()
result["score"] = np.nan


for name, datapoint in datapoints.iterrows():
    score_comprehensive = 0.0

    for parent in datapoint.index:
        score_feature = 1.0
        
        element = datapoint[parent]
        score_feature *= weights[element]

        element = parent
        i = 0
        while True:
            score_feature *= weights[element]

            if i == len(hierarchy.columns) - 1:
                break

            curr_col_name = hierarchy.columns[i]
            next_col_name = hierarchy.columns[i+1] if i < len(hierarchy.columns) else None

            element = hierarchy.loc[hierarchy[curr_col_name] == element, next_col_name].values[0]
            i+=1

        score_comprehensive += score_feature

    result.at[name, "score"] = score_comprehensive

print(result.sort_values(by="score", ascending=False))

                 PricePerDay  Deposit  EnginePower  SailArea  Width     score
Name                                                                         
Laguna 25                190     1000            6        29    274  0.205511
Maxus 24                 250     1000            6        27    255  0.132799
Antila 24                220     2000            4        27    270  0.132643
Antila 24.4 #2           240     1500            6        30    270  0.112955
Aquatic 25T              260     1400            5        31    280  0.110620
Janmor 25                250     3000            6        30    286  0.102568
Antila 24.4              290     1500            6        30    270  0.096458
Mariner 24               290     1500            4        26    252  0.090382
Maxus 24 Evo #2          300     1500            6        30    255  0.079154
Phobos 25                300     2000            6        30    286  0.075675
Antila 28.8              580     2000           10        39    

We can see that rather cheap boats are higher in the ranking as the DM put great weight on this criterion (Money to Technical - 6, PricePerDay - Deposit -> 8)

Laguna 25 is the highest-ranked boat, likely due to a favorable balance of low price per day, reasonable deposit, decent engine power, sail area, and width.

The Phobos 25 #2 and Phobos 24.5rank lowest, likely due to high costs relative to their features.

What is interesting is that thee Antila 28.8 has the highest EnginePower and SailArea but ranks poorly due to its high price (580 per day) and deposit.

## AHP and UTA results comparison

Kendall's tau: 0.3167
Kendall's distance: 82

![](./../plots/uta.png)

![](./../plots/prometheeII.png)

![](./../plots/prometheeI.png)

![](./../plots/ahp.png)

The results from all methods highlighted that Languna 25 is the best choice. This decision is likely due to its lowest price-per-day value compared to the other alternatives. It also accurately reflects reality, as the decision maker is a student—well known for having limited financial resources. However, a difference can be observed in the last position according to the UTA method, which might be due to incomplete or not fully defined preference information.    